### Problem Definition

    <sheet name>
    : patient unique id

        * Ad : adenoma (precursor of a cancer)
        * Sl :
        * CRC : colorectal cancer (colorectal : 대장)

    <data explanation>

    환자마다 갖고 있는 유전자 변이가 다름.
    이에 따라 암세포가 살기 위해서 필요한 물질에 차이가 있음.

    그래서 내가 필요한 것은,
    환자가 어떤 유전자 변이를 갖고 있을 때, 특정 물질에 대한 의존도를 예측하는 모델

    내가 갖고 있는 것
    1. 환자의 유전자 변이 정보
    2. 특정 물질에 대한 의존도

    <goal>

    새로운 샘플의 정보를 넣었을 때, 그 암이 물질에 대한 의존도가 어느정도인지

In [ ]:
#

# Ad : adenoma (precursor of a cancer)
# Sl :
# CRC : colorectal cancer (colorectal : 대장)


# 환자마다 갖고 있는 유전자 변이가 다름.
# 이에 따라 암세포가 살기 위해서 필요한 물질에 차이가 있음.

''' 그래서 내가 필요한 것은,
 환자가 어떤 유전자 변이를 갖고 있을 때, 특정 물질에 대한 의존도를 예측하는 모델

 내가 갖고 있는 것
  1. 환자의 유전자 변이 정보
  2. 특정 물질에 대한 의존도

 > 새로운 샘플의 정보를 넣었을 때, 그 암이 물질에 대한 의존도가 어느정도인지
'''

In [ ]:
'''
 gene symbol : 짧게 쓴 거 (변수 아님)
 gene name : 길게 쓴 거 (변수 아님)
 effect : DNA에 문제가 생기는 형태 (변수일지 아닐지 모름)
 impact : 얼마나 세니? (변수이겠지)
 codon change : DNA가 어떻게 바꼈니? (아마 변수 아님)
 amino acid : DNA가 어떻게 바꼈니? (아마 변수 아님)
 transcript : 만들어지는 단백질 정보 (변수 아님)

 여기부터 다 모름, 공부할게
 rsID :
 HapMap_J :

'''

### Data Info Check

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [36]:
# data path
data_path = '/content/drive/MyDrive/A Colorectal Tumor Organoid Library Demonstrates Progressive Loss of Niche Factor Requirements during Tumorigenesis_supple.xlsx'

# library
import warnings; warnings.filterwarnings('ignore')
import pandas as pd
import openpyxl
import re

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
# loading excel sheets --> takes some time to load
wb = openpyxl.load_workbook(data_path)

> label and more columns added

#### meaning of sheet names

In [20]:
# meaning: number of patients
patient_list = wb.sheetnames
len(patient_list)

55

In [23]:
print(patient_list)

['Ad1', 'Ad2', 'Ad3a', 'Ad3b', 'Ad4a', 'Ad4b', 'Ad5', 'Ad6', 'Ad7', 'Ad8', 'Ad9', 'Ad10', 'SL1a', 'SL1b', 'SL2', 'CRC1a', 'CRC1b', 'CRC2', 'CRC3', 'CRC4', 'CRC5', 'CRC6', 'CRC7', 'CRC8', 'CRC9', 'CRC10', 'CRC11', 'CRC12', 'CRC13', 'CRC14', 'CRC15', 'CRC16', 'CRC17a', 'CRC17b', 'CRC18', 'CRC19', 'CRC20', 'CRC21', 'CRC22', 'CRC23a', 'CRC23b', 'CRC23c', 'CRC24a', 'CRC24b', 'CRC24c', 'CRC25a', 'CRC25b', 'CRC25c', 'CRC26a', 'CRC26b', 'CRC27', 'CRC28', 'CRC29', 'CRC30', 'CRC31']


##### >>>> decided to work with reduced columns <<<<

In [53]:
# too many unique columns
# --> reduced the number of columns


reduced_column_list = ['APC', 'CTNNB1','TCF7L2','FAM123B','SOX9','RNF43',
                       'KRAS','NRAS','BRAF','PIK3CA','TGFR2','ACVR2A','SMAD1',
                       'SMAD3','SMAD4','TP53','FBXW7','MLH1']

# number of reduced columns
len(reduced_column_list)

18

#### concating dataframes

In [59]:
Ad_list = [patient for patient in patient_list if patient.startswith('Ad')] # Ad patient list
SL_list = [patient for patient in patient_list if patient.startswith('SL')] # SL patient list
CRC_list = [patient for patient in patient_list if patient.startswith('CRC')] # CRC patient list

print(Ad_list)
print(SL_list)
print(CRC_list)

['Ad1', 'Ad2', 'Ad3a', 'Ad3b', 'Ad4a', 'Ad4b', 'Ad5', 'Ad6', 'Ad7', 'Ad8', 'Ad9', 'Ad10']
['SL1a', 'SL1b', 'SL2']
['CRC1a', 'CRC1b', 'CRC2', 'CRC3', 'CRC4', 'CRC5', 'CRC6', 'CRC7', 'CRC8', 'CRC9', 'CRC10', 'CRC11', 'CRC12', 'CRC13', 'CRC14', 'CRC15', 'CRC16', 'CRC17a', 'CRC17b', 'CRC18', 'CRC19', 'CRC20', 'CRC21', 'CRC22', 'CRC23a', 'CRC23b', 'CRC23c', 'CRC24a', 'CRC24b', 'CRC24c', 'CRC25a', 'CRC25b', 'CRC25c', 'CRC26a', 'CRC26b', 'CRC27', 'CRC28', 'CRC29', 'CRC30', 'CRC31']


In [62]:
# concating Ad patient data
from tqdm.notebook import tqdm
# made function for concating each patient category excel data
def concat_dataframes(patient_list: list):
    patient_df_list = []
    for patient in tqdm(patient_list):
        patient_df = pd.read_excel(data_path, sheet_name=patient, header=2)
        patient_df['patient_unique_id'] = patient
        patient_df_list.append(patient_df)

    return pd.concat(patient_df_list)

# concating dataframes
Ad_final_df = concat_dataframes(Ad_list)
SL_final_df =concat_dataframes(SL_list)
CRC_final_df = concat_dataframes(CRC_list)

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

In [75]:
# check if each dataframe has any column in reduced column list

# Ad
Ad_all_column_list = []
for Ad_patient in Ad_list:
    Ad_temp_df = Ad_final_df[Ad_final_df['patient_unique_id'] == Ad_patient]
    tmp_list = list(set(Ad_temp_df['Gene_symbole'].unique().tolist()) & set(reduced_column_list))
    print(Ad_patient, '--->', tmp_list)

    Ad_all_column_list += tmp_list
    unique_column_list = set(Ad_all_column_list)
print('-'*150)
print('what Ad patients have from reduced column list --> ', list(unique_column_list))

Ad1 ---> ['APC']
Ad2 ---> ['KRAS', 'APC']
Ad3a ---> ['APC']
Ad3b ---> ['FBXW7', 'APC']
Ad4a ---> ['APC']
Ad4b ---> ['CTNNB1', 'APC']
Ad5 ---> ['APC']
Ad6 ---> ['APC']
Ad7 ---> ['APC']
Ad8 ---> ['KRAS', 'BRAF', 'APC']
Ad9 ---> ['KRAS', 'SOX9', 'APC']
Ad10 ---> ['TCF7L2', 'KRAS', 'APC']
------------------------------------------------------------------------------------------------------------------------------------------------------
what Ad patients have from reduced column list -->  ['SOX9', 'APC', 'FBXW7', 'CTNNB1', 'BRAF', 'KRAS', 'TCF7L2']


In [76]:
# SL
SL_all_column_list = []
for SL_patient in SL_list:
    SL_temp_df = SL_final_df[SL_final_df['patient_unique_id'] == SL_patient]
    tmp_list = list(set(SL_temp_df['Gene_symbole'].unique().tolist()) & set(reduced_column_list))
    print(SL_patient, '--->', tmp_list)

    SL_all_column_list += tmp_list
    unique_column_list = set(SL_all_column_list)
print('-'*150)
print('what Ad patients have from reduced column list --> ', list(unique_column_list))

SL1a ---> ['RNF43', 'BRAF']
SL1b ---> ['RNF43', 'BRAF']
SL2 ---> ['BRAF']
------------------------------------------------------------------------------------------------------------------------------------------------------
what Ad patients have from reduced column list -->  ['RNF43', 'BRAF']


In [81]:
len(CRC_list)

40

In [77]:
# CRC
CRC_all_column_list = []
for CRC_patient in CRC_list:
    CRC_temp_df = CRC_final_df[CRC_final_df['patient_unique_id'] == CRC_patient]
    tmp_list = list(set(CRC_temp_df['Gene_symbole'].unique().tolist()) & set(reduced_column_list))
    print(CRC_patient, '--->', tmp_list)

    CRC_all_column_list += tmp_list
    unique_column_list = set(CRC_all_column_list)
print('-'*150)
print('what Ad patients have from reduced column list --> ', list(unique_column_list))

CRC1a ---> ['TP53', 'BRAF', 'APC']
CRC1b ---> ['KRAS', 'CTNNB1', 'APC']
CRC2 ---> ['TCF7L2', 'KRAS', 'FBXW7', 'APC']
CRC3 ---> ['KRAS', 'NRAS', 'APC']
CRC4 ---> ['KRAS', 'APC']
CRC5 ---> ['TP53', 'SOX9', 'APC']
CRC6 ---> ['TP53', 'APC']
CRC7 ---> ['FBXW7', 'APC']
CRC8 ---> ['PIK3CA', 'KRAS', 'APC']
CRC9 ---> ['TP53', 'FBXW7', 'APC']
CRC10 ---> ['APC']
CRC11 ---> ['PIK3CA', 'KRAS', 'CTNNB1', 'APC']
CRC12 ---> ['SMAD3', 'SMAD1', 'MLH1', 'CTNNB1', 'KRAS', 'TCF7L2']
CRC13 ---> ['PIK3CA', 'ACVR2A', 'RNF43', 'BRAF']
CRC14 ---> ['APC', 'TP53', 'BRAF', 'KRAS', 'TCF7L2']
CRC15 ---> ['ACVR2A', 'FBXW7']
CRC16 ---> ['PIK3CA', 'TP53', 'APC']
CRC17a ---> ['APC', 'ACVR2A', 'RNF43', 'BRAF', 'SMAD4', 'TCF7L2']
CRC17b ---> ['APC', 'TP53', 'ACVR2A', 'RNF43', 'BRAF', 'SMAD4', 'TCF7L2']
CRC18 ---> ['PIK3CA', 'RNF43', 'BRAF']
CRC19 ---> ['TP53']
CRC20 ---> ['TP53', 'KRAS', 'APC']
CRC21 ---> ['TCF7L2']
CRC22 ---> ['TP53', 'SOX9', 'SMAD4', 'APC']
CRC23a ---> ['TP53', 'APC']
CRC23b ---> ['TP53', 'APC']
CRC23c 

In [78]:
print('from CRC: ', len(unique_column_list))
print('from reduced columns: ', len(reduced_column_list))

from CRC:  16
from reduced columns:  18


### Data Readiness Check

In [ ]:
# labels not ready yet